In [217]:
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
import codecs
import os
import re

client = MongoClient()
db = client.world

I downloaded all the archives from CIA World Factbook for the past 20 years. In order to do anything with the data, however, I'll need to parse it from the HTML files, organize it, and put it into a database. In this case, I'll be using MongoDB for its flexibility and ease of use.  
  
Before we can do that though, the files for year 2003 and after were given numbers as names. Since I don't know which topic corresponds to which number, I've chosen to rename all the HTML files to their title. An example is presented below. 

In [ ]:
url = 'factbook_15/fields/2116.html'
f=codecs.open(url, 'r')
soup = BeautifulSoup(f.read(), 'lxml')

In [193]:
soup.find(text='Country').findParent().find_next().text.strip().lower()

'diplomatic representation from the us'

In [158]:
title = soup.find(text=re.compile('Field Listing'))
name = re.search('(?<=Field Listing - ).+', title).group().lower()
re.sub('\s', '_', name) + '.html'

'diplomatic_representation_from_the_us.html'

In [151]:
title

'CIA - The World Factbook 2002 -- Field Listing - Diplomatic representation from the US'

The function below is used for each year to rename the files according to the information given in the file as per the example above. It renames every file in the folder so long as the file is parsible. 

In [194]:
def rename(factbook):
    fact = os.listdir(factbook + '/fields')
    for file in fact:
        try:
            url = factbook + '/fields/' + file
            f = codecs.open(url, 'r')
            soup = BeautifulSoup(f.read(), 'lxml')
            name = soup.find(text='Country').findParent().find_next().text.strip().lower()
            name = re.sub('\s', '_', name) + '.html'
            name = re.sub('/', '_', name)
            os.rename(url, factbook + '/fields/' + name)
        except:
            print(file)

The websites from year to year are all slightly different but mostly the same, so the code to scrape them needs to be slightly adjusted. The code below functions adequately for years 2015-2019 with some minor adjustments necessary between the years. For each year, I run the code below and check each dataframe to evaluate success. If need be, I inspect the HTML and tweak the code a bit.

In [475]:
needs = ['gdp_(purchasing_power_parity)', 'gdp_-_real_growth_rate', 'gdp_-_per_capita_(ppp)', 'natural_resources',
        'exports_-_commodities', 'literacy']

#natural resources

url = 'https://www.cia.gov/library/publications/the-world-factbook/fields/287.html'
f=requests.get(url)
soup = BeautifulSoup(f.text, 'lxml')
table = soup.find('table').find('tbody').findAll('tr')
info_dicts = []
for row in table:
    try:
        data = row.findAll('td')
        name = data[0].text.strip()
        info = data[1].text.strip()
        dict_ = {'country': name, 'resources': info}
        info_dicts.append(dict_)
    except:
        continue
resources_df = pd.DataFrame(info_dicts)

In [476]:
resources_df.head()

,country,resources
0,Afghanistan,"natural gas, petroleum, coal, copper, chromite..."
1,Albania,"petroleum, natural gas, coal, bauxite, chromit..."
2,Algeria,"petroleum, natural gas, iron ore, phosphates, ..."
3,American Samoa,"pumice, pumicite"
4,Andorra,"hydropower, mineral water, timber, iron ore, lead"


In [477]:
#exports

url = 'https://www.cia.gov/library/publications/the-world-factbook/fields/240.html'
f=requests.get(url)
soup = BeautifulSoup(f.text, 'lxml')
table = soup.find('table').find('tbody').findAll('tr')
info_dicts = []
for row in table:
    try:
        data = row.findAll('td')
        name = data[0].text.strip()
        info = data[1].text.strip()
        dict_ = {'country': name, 'exports': info}
        info_dicts.append(dict_)
    except:
        continue
exports_df = pd.DataFrame(info_dicts)

In [478]:
exports_df.head()

,country,exports
0,Afghanistan,"opium, fruits and nuts, handwoven carpets, woo..."
1,Albania,"apparel and clothing, footwear; asphalt, metal..."
2,Algeria,"petroleum, natural gas, and petroleum products..."
3,American Samoa,canned tuna 93%
4,Andorra,"tobacco products, furniture"


In [454]:
#literacy

url = f'https://www.cia.gov/library/publications/resources/the-world-factbook/fields/370.html'
f = requests.get(url)
soup = BeautifulSoup(f.text, 'lxml')
lit_dicts = []
table = soup.find('table').find('tbody').findAll('tr')
for row in table:
    try:
        data = row.findAll('td')
        country = data[0].text.strip()
        lits = data[1].text.strip()
        nums = re.findall('\d+\.?\d?%', lits)
        e = nums[0]
        m = nums[1]
        f = nums[2]
        lit_dict = {'country': country, 'total_literacy': e, 'male_literacy': m, 'female_literacy': f}
        lit_dicts.append(lit_dict)
    except:
        continue
lit_df = pd.DataFrame(lit_dicts)

In [455]:
lit_df.head()

,country,total_literacy,male_literacy,female_literacy
0,Afghanistan,43%,55.5%,29.8%
1,Albania,98.1%,98.5%,97.8%
2,Algeria,81.4%,87.4%,75.3%
3,Andorra,100%,100%,100%
4,Angola,71.1%,82%,60.7%


In [473]:
# gdp

url = f'https://www.cia.gov/library/publications/the-world-factbook/fields/208rank.html'
f=requests.get(url)
soup = BeautifulSoup(f.text, 'lxml')
gdp_dicts = []
table = soup.find('table').find('tbody').findAll('tr')
for row in table:
    try:
        data = row.findAll('td')
        name = row.find('a').text.strip()
        info = data[-2].text.strip()
        dict_ = {'country': name, 'gdp': info}
        gdp_dicts.append(dict_)
    except:
        continue
gdp_df = pd.DataFrame(gdp_dicts)

In [474]:
gdp_df.head()

,country,gdp
0,China,"$25,360,000,000,000"
1,United States,"$19,490,000,000,000"
2,India,"$9,474,000,000,000"
3,Japan,"$5,443,000,000,000"
4,Germany,"$4,199,000,000,000"


In [480]:
# ppp

url = f'https://www.cia.gov/library/publications/the-world-factbook/fields/211rank.html'
f=requests.get(url)
soup = BeautifulSoup(f.text, 'lxml')
ppp_dicts = []
table = soup.find('table').find('tbody').findAll('tr')
for row in table:
    try:
        data = row.findAll('td')
        name = data[1].text.strip()
        info = data[2].text.strip()
        info = re.search('\$\d+\,?\d*\s?\w*', info).group()
        dict_ = {'country': name, 'ppp': info}
        ppp_dicts.append(dict_)
    except:
        continue
ppp_df = pd.DataFrame(ppp_dicts)

In [481]:
ppp_df.head()

,country,ppp
0,Liechtenstein,"$139,100"
1,Qatar,"$124,100"
2,Macau,"$122,000"
3,Monaco,"$115,700"
4,Luxembourg,"$105,100"


In [482]:
# growth

url = f'https://www.cia.gov/library/publications/the-world-factbook/fields/210.html'
f=requests.get(url)
soup = BeautifulSoup(f.text, 'lxml')
growth_dicts = []
table = soup.find('table').find('tbody').findAll('tr')
for row in table:
    try:
        data = row.findAll('td')
        name = data[0].text.strip()
        info = data[1].text.strip()
        info = re.search('-?\d+\.?\d*%', info).group()
        dict_ = {'country': name, 'growth': info}
        growth_dicts.append(dict_)
    except:
        continue
growth_df = pd.DataFrame(growth_dicts)

In [483]:
growth_df.head()

,country,growth
0,Afghanistan,2.7%
1,Albania,3.8%
2,Algeria,1.4%
3,American Samoa,-2.5%
4,Andorra,-1.1%


Now that I have all the tables for all the fields of interest for this project, I merge them all into a single dataframe and remove any NaN entries.

In [498]:
full_df = pd.merge(resources_df, exports_df, on='country', how='outer')
full_df = full_df.merge(lit_df, on='country', how='outer')
full_df = full_df.merge(gdp_df, on='country', how='outer')
full_df = full_df.merge(ppp_df, on='country', how='outer')
full_df = full_df.merge(growth_df, on='country', how='outer')

In [485]:
full_df = full_df.dropna()

In [486]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162 entries, 0 to 264
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   country          162 non-null    object
 1   resources        162 non-null    object
 2   exports          162 non-null    object
 3   total_literacy   162 non-null    object
 4   male_literacy    162 non-null    object
 5   female_literacy  162 non-null    object
 6   gdp              162 non-null    object
 7   ppp              162 non-null    object
 8   growth           162 non-null    object
dtypes: object(9)
memory usage: 12.7+ KB


In [487]:
full_df.head()

,country,resources,exports,total_literacy,male_literacy,female_literacy,gdp,ppp,growth
0,Afghanistan,"natural gas, petroleum, coal, copper, chromite...","opium, fruits and nuts, handwoven carpets, woo...",43%,55.5%,29.8%,"$69,450,000,000","$2,000",2.7%
1,Albania,"petroleum, natural gas, coal, bauxite, chromit...","apparel and clothing, footwear; asphalt, metal...",98.1%,98.5%,97.8%,"$36,010,000,000","$12,500",3.8%
2,Algeria,"petroleum, natural gas, iron ore, phosphates, ...","petroleum, natural gas, and petroleum products...",81.4%,87.4%,75.3%,"$630,000,000,000","$15,200",1.4%
4,Andorra,"hydropower, mineral water, timber, iron ore, lead","tobacco products, furniture",100%,100%,100%,"$3,327,000,000","$49,900",-1.1%
5,Angola,"petroleum, diamonds, iron ore, phosphates, cop...","crude oil, diamonds, refined petroleum product...",71.1%,82%,60.7%,"$193,600,000,000","$6,800",-2.5%


I then add the year for future reference when I'll be analyzing the data

In [489]:
full_df['year'] = 2019

In [490]:
full_df.head(10)

,country,resources,exports,total_literacy,male_literacy,female_literacy,gdp,ppp,growth,year
0,Afghanistan,"natural gas, petroleum, coal, copper, chromite...","opium, fruits and nuts, handwoven carpets, woo...",43%,55.5%,29.8%,"$69,450,000,000","$2,000",2.7%,2019
1,Albania,"petroleum, natural gas, coal, bauxite, chromit...","apparel and clothing, footwear; asphalt, metal...",98.1%,98.5%,97.8%,"$36,010,000,000","$12,500",3.8%,2019
2,Algeria,"petroleum, natural gas, iron ore, phosphates, ...","petroleum, natural gas, and petroleum products...",81.4%,87.4%,75.3%,"$630,000,000,000","$15,200",1.4%,2019
4,Andorra,"hydropower, mineral water, timber, iron ore, lead","tobacco products, furniture",100%,100%,100%,"$3,327,000,000","$49,900",-1.1%,2019
5,Angola,"petroleum, diamonds, iron ore, phosphates, cop...","crude oil, diamonds, refined petroleum product...",71.1%,82%,60.7%,"$193,600,000,000","$6,800",-2.5%,2019
8,Antigua and Barbuda,NEGL; pleasant climate fosters tourism,"petroleum products, bedding, handicrafts, elec...",99%,98.4%,99.4%,"$2,398,000,000","$26,400",2.8%,2019
10,Argentina,"fertile plains of the pampas, lead, zinc, tin,...","soybeans and derivatives, petroleum and gas, v...",99%,98.9%,99.1%,"$922,100,000,000","$20,900",2.9%,2019
11,Armenia,"small deposits of gold, copper, molybdenum, zi...","unwrought copper, pig iron, nonferrous metals,...",99.7%,99.8%,99.7%,"$28,340,000,000","$9,500",7.5%,2019
12,Aruba,NEGL; white sandy beaches foster tourism,"live animals and animal products, art and coll...",97.8%,97.8%,97.8%,"$4,158,000,000","$37,500",1.2%,2019
17,Azerbaijan,"petroleum, natural gas, iron ore, nonferrous m...","oil and gas roughly 90%, machinery, foodstuffs...",99.8%,99.9%,99.7%,"$172,200,000,000","$17,500",0.1%,2019


In [178]:
def per_float(percent):
    '''This accepts a percentage object and converts it into a decimal float'''
    percent = re.sub('%', '', percent)
    return round(float(percent)/100, 3)

In [318]:
full_df.columns

Index(['country', 'resources', 'exports', 'total_literacy', 'male_literacy',
       'female_literacy', 'gdp', 'ppp', 'growth', 'year', 'femal_literacy'],
      dtype='object')

In [491]:
full_df['total_literacy'] = full_df['total_literacy'].apply(per_float)
full_df['male_literacy'] = full_df['male_literacy'].apply(per_float)
full_df['female_literacy'] = full_df['female_literacy'].apply(per_float)
full_df['growth'] = full_df['growth'].apply(per_float)

In [211]:
def dollar_int(dollar):
    '''This accepts a dollar object and converts into an integer'''
    dollar = re.sub(',','',dollar)
    return int(dollar[1:])

In [492]:
full_df['ppp'] = full_df['ppp'].apply(dollar_int)

In [340]:
def gdp_int(gdp):
    '''GDP is commonly displayed in terms of billions and trillions. 
    This accepts the written object amount and converts it into an integer'''
    num = float(re.search('\d+\.?\d*', gdp).group())
    power = re.search('(million|billion|trillion)', gdp).group()
    if power == 'billion':
        num *= 10**9
    elif power == 'million':
        num *= 10**6
    elif power == 'trillion':
        num *= 10**12
    return int(num)

In [420]:
full_df['gdp'] = full_df['gdp'].apply(gdp_int)

After completing all the conversions, the final dataframe has all the data in the correct types

In [493]:
full_df.head(10)

,country,resources,exports,total_literacy,male_literacy,female_literacy,gdp,ppp,growth,year
0,Afghanistan,"natural gas, petroleum, coal, copper, chromite...","opium, fruits and nuts, handwoven carpets, woo...",0.430,0.555,0.298,69450000000,2000,0.027,2019
1,Albania,"petroleum, natural gas, coal, bauxite, chromit...","apparel and clothing, footwear; asphalt, metal...",0.981,0.985,0.978,36010000000,12500,0.038,2019
2,Algeria,"petroleum, natural gas, iron ore, phosphates, ...","petroleum, natural gas, and petroleum products...",0.814,0.874,0.753,630000000000,15200,0.014,2019
4,Andorra,"hydropower, mineral water, timber, iron ore, lead","tobacco products, furniture",1.000,1.000,1.000,3327000000,49900,-0.011,2019
5,Angola,"petroleum, diamonds, iron ore, phosphates, cop...","crude oil, diamonds, refined petroleum product...",0.711,0.820,0.607,193600000000,6800,-0.025,2019
8,Antigua and Barbuda,NEGL; pleasant climate fosters tourism,"petroleum products, bedding, handicrafts, elec...",0.990,0.984,0.994,2398000000,26400,0.028,2019
10,Argentina,"fertile plains of the pampas, lead, zinc, tin,...","soybeans and derivatives, petroleum and gas, v...",0.990,0.989,0.991,922100000000,20900,0.029,2019
11,Armenia,"small deposits of gold, copper, molybdenum, zi...","unwrought copper, pig iron, nonferrous metals,...",0.997,0.998,0.997,28340000000,9500,0.075,2019
12,Aruba,NEGL; white sandy beaches foster tourism,"live animals and animal products, art and coll...",0.978,0.978,0.978,4158000000,37500,0.012,2019
17,Azerbaijan,"petroleum, natural gas, iron ore, nonferrous m...","oil and gas roughly 90%, machinery, foodstuffs...",0.998,0.999,0.997,172200000000,17500,0.001,2019


In [494]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162 entries, 0 to 264
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   country          162 non-null    object 
 1   resources        162 non-null    object 
 2   exports          162 non-null    object 
 3   total_literacy   162 non-null    float64
 4   male_literacy    162 non-null    float64
 5   female_literacy  162 non-null    float64
 6   gdp              162 non-null    int64  
 7   ppp              162 non-null    int64  
 8   growth           162 non-null    float64
 9   year             162 non-null    int64  
dtypes: float64(4), int64(3), object(3)
memory usage: 13.9+ KB


Next, I iterate through the rows of the data to insert each into my Mongo Database for easy future retrieval. 

In [495]:
for row in full_df.itertuples():
    country_dict = {'_id': row[1]+str(row[-1]), 'country': row[1], 'year': row[-1], 'gdp': row[7], 'ppp': row[8], 
                    'growth': row[9], 'resources': row[2], 'exports': row[3], 'literacy': row[4], 'male': row[5],
                   'female': row[6]}
    db.resources.insert_one(country_dict)